In [1]:
sc

<SparkContext master=yarn appName=PySparkShell>

In [2]:
sc.master

'yarn'

In [86]:
sc.setCheckpointDir('hdfs://zh:9000/checkpoint')

In [113]:
from pyspark.mllib.recommendation import ALS, Rating
from pyspark.mllib.clustering import KMeans
from pyspark.mllib.linalg import Vectors
import numpy as np

### 数据读取

In [4]:
movies = sc.textFile('hdfs://zh:9000/ml-100k/u.item')

In [5]:
movies.take(1)

['1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0']

In [6]:
genres = sc.textFile('hdfs://zh:9000/ml-100k/u.genre')

In [15]:
genres.take(101)

['unknown|0',
 'Action|1',
 'Adventure|2',
 'Animation|3',
 "Children's|4",
 'Comedy|5',
 'Crime|6',
 'Documentary|7',
 'Drama|8',
 'Fantasy|9',
 'Film-Noir|10',
 'Horror|11',
 'Musical|12',
 'Mystery|13',
 'Romance|14',
 'Sci-Fi|15',
 'Thriller|16',
 'War|17',
 'Western|18',
 '']

* 注意最后有一个是空的''

In [54]:
genresMap = genres.filter(lambda x:x!='').map(lambda x:x.split('|'), ).map(lambda x:(x[1], x[0]))

In [280]:
genres_dict={i:j for i, j in genresMap.collect()}

In [281]:
genres_dict

{'0': 'unknown',
 '1': 'Action',
 '10': 'Film-Noir',
 '11': 'Horror',
 '12': 'Musical',
 '13': 'Mystery',
 '14': 'Romance',
 '15': 'Sci-Fi',
 '16': 'Thriller',
 '17': 'War',
 '18': 'Western',
 '2': 'Adventure',
 '3': 'Animation',
 '4': "Children's",
 '5': 'Comedy',
 '6': 'Crime',
 '7': 'Documentary',
 '8': 'Drama',
 '9': 'Fantasy'}

In [282]:
sc.broadcast(genres_dict)

In [283]:
movies.take(1)

['1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0']

In [284]:
Binary_movies_genres =  movies.map(
    lambda x:x.split('|')
    ).map(
    lambda x:(x[0], (x[1], [genres_dict.get(str(i)) for i, j in list(enumerate(x[5:])) if j == "1"]))
    )

In [285]:
Binary_movies_genres.take(2)

[('1', ('Toy Story (1995)', ['Animation', "Children's", 'Comedy'])),
 ('2', ('GoldenEye (1995)', ['Action', 'Adventure', 'Thriller']))]

### 训练获得用户隐式因子和电影隐式因子

In [76]:
rawData = sc.textFile('hdfs://zh:9000/ml-100k/u.data')

In [77]:
rawData.take(10)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596',
 '298\t474\t4\t884182806',
 '115\t265\t2\t881171488',
 '253\t465\t5\t891628467',
 '305\t451\t3\t886324817',
 '6\t86\t3\t883603013']

In [78]:
rawRatings = rawData.map(lambda x:x.split('\t')) 

In [79]:
rawRatings.take(5)

[['196', '242', '3', '881250949'],
 ['186', '302', '3', '891717742'],
 ['22', '377', '1', '878887116'],
 ['244', '51', '2', '880606923'],
 ['166', '346', '1', '886397596']]

In [81]:
ratings = rawRatings.map(lambda x:Rating(x[0], x[1], x[2]))

In [82]:
ratings.take(2)

[Rating(user=196, product=242, rating=3.0),
 Rating(user=186, product=302, rating=3.0)]

In [87]:
ratings.cache()
ratings.checkpoint()

In [91]:
alsModel = ALS.train(ratings, 50, 10, 0.1)

In [133]:
movieFactors = alsModel.productFeatures().map(lambda x:(x[0], Vectors.dense(x[1])))

In [134]:
movieFactors.take(1)

[(2,
  DenseVector([0.3654, -0.173, -0.1281, -0.3246, 0.3738, -0.1943, 0.164, -0.3877, -0.0255, 0.4857, 0.4512, 0.3212, -0.0766, -0.1516, -0.1754, 0.1534, -0.0947, 0.2612, 0.1665, -0.1016, -0.4541, 0.3292, 0.5282, 0.1369, -0.2008, -0.2896, -0.0409, -0.0477, 0.6863, 0.144, 0.2097, -0.1229, 0.3868, 0.1818, 0.0641, 0.0938, -0.0125, -0.1375, -0.0872, -0.1569, -0.1571, 0.1495, -0.0237, 0.0604, 0.091, 0.0002, -0.2266, 0.2357, 0.296, -0.3055]))]

In [135]:
movieVectors = movieFactors.map(lambda x:x[1])

In [136]:
userFactors = alsModel.userFeatures().map(lambda x:(x[0], Vectors.dense(x[1])))

In [137]:
userVectors = userFactors.map(lambda x:x[1])

In [132]:
userFactors.take(1)

[(2,
  DenseVector([0.3655, -0.072, 0.1708, -0.6981, 0.3765, -0.0557, 0.0704, -0.2848, 0.3052, 0.3762, 0.4244, 0.7517, -0.1863, -0.1985, -0.3376, 0.192, 0.0647, 0.3941, -0.343, 0.1128, -0.5409, 0.4214, 0.4978, 0.0817, -0.0997, -0.5185, 0.2245, -0.2071, 0.699, -0.1033, 0.2453, -0.039, 0.082, 0.2907, 0.1014, 0.6009, 0.1217, 0.1533, -0.5877, -0.2777, -0.1037, 0.121, -0.1382, -0.0424, 0.3514, -0.2184, -0.0749, 0.4529, 0.2757, 0.0583]))]

In [139]:
userVectors.take(1)

[DenseVector([0.3655, -0.072, 0.1708, -0.6981, 0.3765, -0.0557, 0.0704, -0.2848, 0.3052, 0.3762, 0.4244, 0.7517, -0.1863, -0.1985, -0.3376, 0.192, 0.0647, 0.3941, -0.343, 0.1128, -0.5409, 0.4214, 0.4978, 0.0817, -0.0997, -0.5185, 0.2245, -0.2071, 0.699, -0.1033, 0.2453, -0.039, 0.082, 0.2907, 0.1014, 0.6009, 0.1217, 0.1533, -0.5877, -0.2777, -0.1037, 0.121, -0.1382, -0.0424, 0.3514, -0.2184, -0.0749, 0.4529, 0.2757, 0.0583])]

### 训练聚类模型

In [106]:
numClusters = 5

In [107]:
numIteration = 10

In [108]:
numRuns = 3

In [140]:
moviesClusterModel = KMeans.train(movieVectors, numClusters, maxIterations=numIteration)

### 使用聚类模型进行预测

In [142]:
moviesClusterModel.predict(movieVectors.first())

4

### 解释电影类簇

In [256]:
centers_list = moviesClusterModel.clusterCenters

In [257]:
centers_dict = { i:j for i,j in enumerate(centers_list)}

In [258]:
centers_dict

{0: array([ 0.38719752,  0.02032083, -0.10446179, -0.27967196,  0.3644459 ,
        -0.00908174,  0.06710843, -0.23035321,  0.18922924,  0.37817816,
         0.41319196,  0.48445528,  0.01535765, -0.06402941, -0.22976315,
         0.1352709 ,  0.14354886,  0.39591354,  0.01759195, -0.09645237,
        -0.53708381,  0.39901285,  0.25927439,  0.03715001,  0.11915951,
        -0.24101959,  0.0525625 , -0.43471468,  0.48108509, -0.04809547,
         0.03144005,  0.06208569,  0.02790381,  0.17943966,  0.03559354,
         0.34086505, -0.04297451, -0.14302302, -0.32548359, -0.28540418,
        -0.25501198, -0.05129134, -0.04102628, -0.0691379 ,  0.3225153 ,
        -0.02328049, -0.08112066,  0.37689738,  0.17636683,  0.13715905]),
 1: array([  1.73111366e-01,  -3.90677871e-02,   1.78055008e-02,
         -1.56068935e-01,   6.37029722e-02,  -3.07082517e-02,
          8.51927636e-03,  -9.11058007e-02,   1.16210625e-04,
          1.31485224e-01,   6.61308834e-02,   1.81334354e-01,
         -9.23

In [259]:
sc.broadcast(centers_dict)

In [260]:
cluster_movie = moviesClusterModel.predict(movieVectors).zip(movieFactors)

In [261]:
cluster_movie.take(1)

[(4,
  (2,
   DenseVector([0.3654, -0.173, -0.1281, -0.3246, 0.3738, -0.1943, 0.164, -0.3877, -0.0255, 0.4857, 0.4512, 0.3212, -0.0766, -0.1516, -0.1754, 0.1534, -0.0947, 0.2612, 0.1665, -0.1016, -0.4541, 0.3292, 0.5282, 0.1369, -0.2008, -0.2896, -0.0409, -0.0477, 0.6863, 0.144, 0.2097, -0.1229, 0.3868, 0.1818, 0.0641, 0.0938, -0.0125, -0.1375, -0.0872, -0.1569, -0.1571, 0.1495, -0.0237, 0.0604, 0.091, 0.0002, -0.2266, 0.2357, 0.296, -0.3055])))]

In [262]:
cluster_movie.cache()
cluster_movie.checkpoint()

In [263]:
def dist_vector(x_array, y_array):
    return np.linalg.norm(x_array - y_array)

In [264]:
cluster_movie_dist = cluster_movie.map(lambda x:(x[0], (x[1][0], dist_vector(centers_dict[x[0]], x[1][1])) ))

In [265]:
cluster_movie_dist.take(1)

[(4, (2, 0.89839991944878539))]

In [266]:
cluster_movie_dist_sorted = cluster_movie_dist.sortBy(keyfunc=lambda x:x[1][1]).sortByKey()

In [267]:
cluster_movie_dist_sorted.take(10)

[(0, (1123, 0.42600146324665555)),
 (0, (1526, 0.43585271409902199)),
 (0, (1223, 0.52027197896886157)),
 (0, (1506, 0.57875724022966224)),
 (0, (1674, 0.58303115473767486)),
 (0, (1682, 0.59244316581429246)),
 (0, (1454, 0.60002032054745158)),
 (0, (1537, 0.61113793622031243)),
 (0, (1541, 0.63662497458051748)),
 (0, (1573, 0.66317916453326287))]

In [278]:
cluster_movie_dist.cache()
cluster_movie_dist.checkpoint()

In [286]:
top_n = 20
for i in range(5):
    print("cluster:%s"%i)
    top_n_i = cluster_movie_dist.filter(
    lambda x:x[0]==i
    ).sortBy(
    keyfunc=lambda x:x[1][1]
    ).map(
    lambda x:x[1]
    ).join(
    Binary_movies_genres.map(lambda x:(int(x[0]),x[1]))
    ).take(top_n)
    genres_counts_dict = {} 
    for movie_item in top_n_i:
        print(movie_item)
        genres_types = movie_item[1][1][1]
        for movie_type in genres_types:
            genres_counts_dict[movie_type] = genres_counts_dict.get(movie_type, 0) + 1
    print(genres_counts_dict)
    print('============================')

cluster:0
(1024, (1.2076547455154274, ('Mrs. Dalloway (1997)', ['Romance'])))
(512, (1.4392482878048338, ('Wings of Desire (1987)', ['Comedy', 'Drama', 'Romance'])))
(4, (0.99306995380880725, ('Get Shorty (1995)', ['Action', 'Comedy', 'Drama'])))
(8, (1.1938202481575975, ('Babe (1995)', ["Children's", 'Comedy', 'Drama'])))
(12, (0.99214063365323968, ('Usual Suspects, The (1995)', ['Crime', 'Thriller'])))
(20, (1.3916108617423153, ('Angels and Insects (1995)', ['Drama', 'Romance'])))
(32, (1.1834490100311863, ('Crumb (1994)', ['Documentary'])))
(1200, (0.8044069056113895, ('Kim (1950)', ["Children's", 'Drama'])))
(48, (1.0828354841508447, ('Hoop Dreams (1994)', ['Documentary'])))
(52, (0.74546665301676751, ('Madness of King George, The (1994)', ['Drama'])))
(56, (1.299995432864169, ('Pulp Fiction (1994)', ['Crime', 'Drama'])))
(180, (1.0508807331593284, ('Apocalypse Now (1979)', ['Drama', 'War'])))
(60, (1.1836726112218896, ('Three Colors: Blue (1993)', ['Drama'])))
(1100, (1.3855060190

In [296]:
# (1104, (1.640930114469334, ("C'est arriv� pr�s de chez vous (1992)", [None, None, None])))
# (1108, (1.1315929580238773, ('Feast of July (1995)', [None])))
# (1116, (1.2182542031236385, ('Mark of Zorro, The (1940)', [None])))
# (1120, (1.5608904630993607, ("I'm Not Rappaport (1996)", [None])))
# (1124, (0.67582726573118568, ('Farewell to Arms, A (1932)', [None, None])))      
# {None: 36}     
# cluster:1
# (1672, (0.58521085647527527, ('Kika (1993)', [None])))
# (1676, (0.39349518422471019, ('War at Home, The (1996)', [None])))

* genres_dict之前定义为0-1变量的index和电影genre的对应，但是这里变成了genres_dict = {}。
* Binary_movies_genres是之前的结果，可能依赖了之前的genres_dict，结果导致Binary_movies_genres变了，join出的结果也都显示类型为None

### WCSS

In [288]:
train_, test_ = movieVectors.randomSplit([0.6, 0.4])

In [292]:
k_list = [1, 2, 3, 4, 5, 6, 7, 9, 10,11]

In [297]:
costsuser = 0
for k in k_list:
    previous = costsuser
    costsuser = KMeans.train(train_, k).computeCost(test_)
    print("k:%s ==> WCSS:%s ==> edge:%s"%(k, costsuser, costsuser-previous))

k:1 ==> WCSS:1103.11988808383 ==> edge:1103.11988808383
k:2 ==> WCSS:991.3102971737878 ==> edge:-111.80959091004206
k:3 ==> WCSS:957.4243015933259 ==> edge:-33.885995580461895
k:4 ==> WCSS:934.9998894619946 ==> edge:-22.424412131331337
k:5 ==> WCSS:916.1458577408989 ==> edge:-18.85403172109568
k:6 ==> WCSS:917.2173198414066 ==> edge:1.0714621005076879
k:7 ==> WCSS:889.0006071657153 ==> edge:-28.21671267569127
k:9 ==> WCSS:868.3955560697436 ==> edge:-20.605051095971703
k:10 ==> WCSS:874.3546476642673 ==> edge:5.95909159452367
k:11 ==> WCSS:866.0871260856904 ==> edge:-8.267521578576861
